In [9]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
import itertools

In [10]:
# Loading the necessary data
train_df = pd.read_csv("fixture_training_data.csv")
fixtures_df = pd.read_csv("2526_team-fixtures.csv")

# Getting each individual team for the 2025-26 season
teams_2526 = pd.unique(fixtures_df[["team", "opponent"]].values.ravel())
train_df = train_df[train_df["team"].isin(teams_2526) & train_df["opponent"].isin(teams_2526)].copy()

# Setting feature and target columns
feature_columns = ["xG", "Poss", "Standard Sh", "Standard SoT"] #expected goals, possessions, shots and shots on target
target_columns = ["GF", "GA"] # goals for and goals against

# Sorting the training data by team and date
train_df["date"] = pd.to_datetime(train_df["date"], dayfirst=True)
train_df = train_df.sort_values(["team", "date"])

# Calculating rolling averages (last 5 games) for each team 
for col in feature_columns + target_columns:
    train_df[f"{col}_roll5"] = (
        train_df.groupby("team")[col]
        .transform(lambda x: x.shift().rolling(5, min_periods=1).mean())
    )


# Creating opponent features and preparing data for training/testing
opp_df = train_df[["team","date"] + [f"{c}_roll5" for c in feature_columns]].copy()
opp_df = opp_df.rename(columns={"team":"opponent", **{f"{c}_roll5":f"opp_{c}_roll5" for c in feature_columns}})
train_df = train_df.merge(opp_df, on=["opponent","date"], how="left")

train_df = train_df.dropna(subset=[f"{c}_roll5" for c in feature_columns] +
                           [f"opp_{c}_roll5" for c in feature_columns])

X = train_df[[f"{c}_roll5" for c in feature_columns] +
             [f"opp_{c}_roll5" for c in feature_columns]]
y = train_df[target_columns]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 42)

# Training and fitting the XGBRegressor on the training data
regressor = MultiOutputRegressor(XGBRegressor(objective = "reg:squarederror", n_estimators = 500, max_depth = 4, random_state = 42))
regressor.fit(X_train, y_train)

# Calculating the root mean squared error (lower is better :D)
print("Train RMSE:", root_mean_squared_error(y_train, regressor.predict(X_train)))
print("Test RMSE:", root_mean_squared_error(y_test, regressor.predict(X_test)))

# Calculating team strength coefficients using their average goals scored/conceded (for fairness)
team_strength = train_df.groupby("team").agg({"GF":"mean", "GA": "mean"}).reset_index()
league_average_gf = team_strength["GF"].mean()
league_average_ga = team_strength["GA"].mean()

team_strength["attacking_strength"] = team_strength["GF"] / league_average_gf
team_strength["defensive_strength"] = team_strength["GA"] / league_average_ga

# Including home advantage (for fairness as well)
home_advantage = train_df.groupby("venue").agg({"GF": "mean"}).reset_index()

if "Home" in home_advantage["venue"].values:
    home_advantage_multiplier = home_advantage.loc[home_advantage["venue"] == "Home", "GF"].values[0] / train_df["GF"].mean()

else:
    home_advantage_multiplier = 1.1

strength_dictionary = team_strength.set_index("team")[["attacking_strength", "defensive_strength"]].to_dict(orient="index")

# Handling the promoted teams for the 2025-26 season (1. FC Koln and Hamburger SV)
koln_stats = train_df.loc[train_df["team"] == "Koln", ["GF", "GA"]].mean().to_dict() # use Koln's last available stats
strength_dictionary["Koln"] = {
    "attacking_strength": koln_stats["GF"] / league_average_gf,
    "defensive_strength": koln_stats["GA"] / league_average_ga
}

bottom_3 = train_df.groupby("team")["GF"].mean().nsmallest(3).index # using the average stats of the three teams with the lowest number of goals
hamburger_sv_stats = train_df.loc[train_df["team"].isin(bottom_3), ["GF", "GA"]].mean().to_dict()
strength_dictionary["Hamburger SV"] = {
    "attacking_strength": hamburger_sv_stats["GF"] / league_average_gf,
    "defensive_strength": hamburger_sv_stats["GA"] / league_average_ga
}

# Creating a match schedule by pairing teams (total of 306 games) and adding it to the full_schedule list
schedule = list(itertools.combinations(teams_2526, 2))

full_schedule = []

for home, away in schedule:
    full_schedule.append({"team": home, "opponent": away, "venue": "Home"})
    full_schedule.append({"team": away, "opponent": home, "venue": "Away"})


# Running a single season and generating match predictions
def run_single_season(strength_dictionary, full_schedule, home_advantage, league_average_gf, league_average_ga):
    season_results = []
    all_fixtures_predictions = []

    for game in full_schedule:
        home = game["team"]
        away = game["opponent"]
        home_strength = strength_dictionary.get(home, {"attacking_strength": 1, "defensive_strength": 1})
        away_strength = strength_dictionary.get(away, {"attacking_strength": 1, "defensive_strength": 1})

        # Custom defined formulas for home and away team expected goals (including home advantage)
        home_xg = home_strength["attacking_strength"] * away_strength["defensive_strength"] * league_average_gf * home_advantage
        away_xg = away_strength["attacking_strength"] * home_strength["defensive_strength"] * league_average_ga

        # Poisson distribution for goal prediction (introduces randomness)
        home_gf = np.random.poisson(lam = home_xg)
        away_gf = np.random.poisson(lam = away_xg)

        # Generating the fixture score predictions
        all_fixtures_predictions.append({
            "Home": home,
            "Away": away,
            "Predicted Score": f"{home_gf} - {away_gf}"
        })

        if home_gf > away_gf:
            home_points, away_points = 3, 0
        
        elif home_gf < away_gf:
            home_points, away_points = 0, 3
        
        else:
            home_points, away_points = 1, 1
        
        season_results.append([home, home_points, home_gf, away_gf])
        season_results.append([away, away_points, away_gf, home_gf])
        
    season_results_df = pd.DataFrame(season_results, columns=["team", "points", "gf", "ga"])
    
    # Generating match outcomes based on predicted fixtures
    def get_match_outcome(row):
        if row["gf"] > row["ga"]:
            return "W"
        
        elif row["gf"] < row["ga"]:
            return "L"
        
        else:
            return "D"
        
    season_results_df['outcome'] = season_results_df.apply(get_match_outcome, axis=1)

    # Generating the league table
    table = season_results_df.groupby("team").agg(
        games_played = ("team", "size"),
        wins = ("outcome", lambda x: (x == "W").sum()),
        draws = ("outcome", lambda x: (x == "D").sum()),
        losses = ("outcome", lambda x: (x == "L").sum()),
        points = ("points", "sum"),
        gf = ("gf", "sum"),
        ga = ("ga", "sum"),
    
    ).reset_index()
    
    table['gd'] = table['gf'] - table['ga'] # goal difference
    table = table.sort_values(by=["points", "gd", "gf"], ascending=False).reset_index(drop=True)
    table["position"] = range(1, len(table) + 1) # position

    return table[['team', 'position', 'games_played', 'wins', 'draws', 'losses', 'points', 'gf', 'ga', 'gd']], pd.DataFrame(all_fixtures_predictions)


# Running the predictor for 1000 simulations
num_simulations = 1000
position_counts = {team: {pos: 0 for pos in range(1, len(teams_2526) + 1)} for team in teams_2526}

print(f"Running {num_simulations} simulations...")
for i in range(num_simulations):
    np.random.seed(i)

    final_table, fixtures = run_single_season(strength_dictionary, full_schedule, home_advantage = 1.1, league_average_gf = league_average_gf, league_average_ga = league_average_ga)

    # Fixture scores for the first simulation
    if i == 0:
        print("\nPredicted Fixture Scores for the First Simulation:")
        fixtures_df = pd.DataFrame(fixtures)
        fixtures_df.to_csv("first_sim_predictions_2526.csv", index=False)
        print(fixtures_df.to_string(index=False))
    
    for _, row in final_table.iterrows():
        team = row['team']
        position = row['position']
        position_counts[team][position] += 1 / num_simulations * 100
        position_counts[team][position] = round(position_counts[team][position], 1)

# Generating the final table
results_df = pd.DataFrame.from_dict(position_counts, orient='index')
results_df.index.name = 'Team'
results_df.columns.name = 'Position'

results_df = results_df.sort_index()

print("\nFinal Position Probability Grid (after 1000 simulations) (all values are in percent):")
print(results_df.to_string())

results_df.to_csv("2526_position_grid.csv")




Train RMSE: 0.14615345001220703
Test RMSE: 1.4764893054962158
Running 1000 simulations...

Predicted Fixture Scores for the First Simulation:
               Home                Away Predicted Score
           Augsburg            Freiburg           2 - 2
           Freiburg            Augsburg           3 - 2
           Augsburg       Bayern Munich           1 - 1
      Bayern Munich            Augsburg           7 - 1
           Augsburg            St Pauli           2 - 0
           St Pauli            Augsburg           1 - 1
           Augsburg            Mainz 05           2 - 3
           Mainz 05            Augsburg           1 - 2
           Augsburg          Heidenheim           0 - 1
         Heidenheim            Augsburg           3 - 0
           Augsburg           Wolfsburg           0 - 1
          Wolfsburg            Augsburg           1 - 0
           Augsburg                Koln           0 - 1
               Koln            Augsburg           0 - 2
           Augsbur